# Quiz 6 - Object Recognition: BoF vs ConvNets
#### Edson Roteia Araujo Junior e João Pedro Moreira Ferreira

### Instructions
The goal of this quiz is to implement two object recognition approaches:

1.     A classifier based on bag of features:

    - Use SVM or Random Forest as classifiers.

    - Try different sizes for the dictionary.

2.     A classifier using ConvNet implemented in Keras:
 
     - Use an architecture inspired in the LeNet5

Your code must be implemented on a notebook python and you must use the CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.html) for training and testing.

The notebook must present a confusion matrix and the average accuracy for each approach. You also have to report both training and test accuracies.



### Bag of Features ###

#### In order to enable the SIFT module of the OpenCV library.<span style="color:red"> We install the opencv-contrib-python at version 3.4.2.16.</span> You should install it to make this notebook work. You can install it through any python package manager, since the module is in a pypi repository. More informations [here](https://pypi.org/project/opencv-contrib-python/3.4.2.16/) ####

#### Also you may need to install the <span style="color:red">scikit-image</span> module. Again it can be installed by your python package manager ####

In [1]:
!pip install opencv-contrib-python==3.4.2.16

### Load required Libraries

In [23]:
import numpy as np
import cv2 
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix
from keras.datasets import cifar10
from keras.utils import np_utils
from skimage import img_as_ubyte
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [25]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train_not_categorical = y_train.copy()
y_train = np_utils.to_categorical(y_train)
y_test_not_categorical = y_test.copy()
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

print("Train Data Size: %d" % len(X_train))
print("Test Data Size: %d" % len(X_test))
print("Number of Classes: %d" % num_classes)

Train Data Size: 50000
Test Data Size: 10000
Number of Classes: 10


### Extracting Features with SIFT



In [4]:
def featureExtraction(sift,image):
    image = img_as_ubyte(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors
sift = cv2.xfeatures2d.SIFT_create()

#### Training Features

In [5]:
train_descriptors_list = []
for image in X_train:
    keypoints, descriptors = featureExtraction(sift,image)
    train_descriptors_list.append(descriptors)
    
print("Number of training images with descriptors: %d" % sum(x is not None for x in train_descriptors_list))

Number of training images with descriptors: 49906


#### Test Features

In [6]:
test_descriptors_list = []
for image in X_test:
    keypoints, descriptors = featureExtraction(sift,image)
    test_descriptors_list.append(descriptors)
    
print("Number of test images with descriptors: %d" % sum(x is not None for x in test_descriptors_list))

Number of test images with descriptors: 9986


### Clustering

In [7]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

In [8]:
flatten_descriptors_list = []
for sublist in train_descriptors_list:
    if not sublist is None:  
      for item in sublist:
          flatten_descriptors_list.append(item)

print("Number of descriptors in total: %d" % len(flatten_descriptors_list))

Number of descriptors in total: 656461


In [9]:
def get_bovws(k):
  kmeans = MiniBatchKMeans(n_clusters = k, verbose = False, max_iter = 5000, batch_size = 500)
  kmeans.fit(flatten_descriptors_list)
  
  train_image_bags = []
  for image_descriptors in train_descriptors_list:
    if(image_descriptors is not None):
      histogram = build_histogram(image_descriptors, kmeans)
      train_image_bags.append(histogram)
    else:
      train_image_bags.append(np.zeros((k,)))
  
  test_image_bags = []
  for image_descriptors in test_descriptors_list:
    if(image_descriptors is not None):
      histogram = build_histogram(image_descriptors, kmeans)
      test_image_bags.append(histogram)
    else:
      test_image_bags.append(np.zeros((k,)))
  
  return train_image_bags, test_image_bags


In [15]:
k = 3
train_image_bags, test_image_bags = get_bovws(k)
print(train_image_bags[0:5])
print(test_image_bags[0:5])

[array([8., 0., 6.]), array([8., 8., 5.]), array([2., 4., 7.]), array([9., 0., 4.]), array([4., 4., 6.])]
[array([6., 0., 4.]), array([3., 6., 4.]), array([ 1., 10.,  1.]), array([2., 2., 4.]), array([8., 2., 5.])]


In [ ]:
bow = cv2.BOWKMeansTrainer(100)
for desc in flatten_descriptors_list:
    bow.add(desc)
print 'Added all to BoW'
dictionary = bow.cluster()
print 'Created clusters'

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = {}  # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)

bowDiction = cv2.BOWImgDescriptorExtractor(cv2.xfeatures2d.SIFT_create(), flann)
bowDiction.setVocabulary(dictionary)


### Random Forest Classifier

In [29]:
rfc = RandomForestClassifier()
rfc.fit(train_image_bags, y_train)
#acc = rfc.score(test_image_bags, y_test)
acc = rfc.score(train_image_bags, y_train)
print("Random Forest Classifier accuracy: {:.2f}%".format(acc * 100))
predictions = rfc.predict(test_image_bags)
conf_matrix = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
print(conf_matrix)

Random Forest Classifier accuracy: 2.86%
[[969   1   1   3   1   4   6   3   8   4]
 [967   5   0   0   0   3   9   7   1   8]
 [981   2   1   1   1   3   1   3   2   5]
 [965   2   0   4   2   6   5   5   3   8]
 [970   3   1   3   1   3   5   5   5   4]
 [953   1   0   8   4  14  10   6   0   4]
 [963   3   1   2   1   6   6   4   1  13]
 [951   2   1   8   1  12   4   9   3   9]
 [975   0   0   1   1   2   2   0  17   2]
 [952   5   0   9   1   5   0  12   2  14]]


### SVM Classifier

In [ ]:
# clf = SVC()
# clf.fit(image_bags, y_train_not_categorical)
# acc = clf.score(image_bags, y_train_not_categorical)
# print("[INFO] SVM search accuracy: {:.2f}%".format(acc * 100))

### ConvNet ###

In [ ]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from PIL import Image
# from scipy.misc import toimage -> DEPRECATED
from keras.layers import *

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# create a grid of 3x3 images
for i in range(0, 9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(Image.fromarray(X_train[i]))
# show the plot
pyplot.show()

#### Simple Covolutional Neural Network for CIFAR-10 ####

In [ ]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.layers import *
K.set_image_dim_ordering('th')

seed = 7
numpy.random.seed(seed)

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


#### Larger Covolutional Neural Network for CIFAR-10 ####

In [ ]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

#### Architecture Inspired in the LeNet5 ####

![lenet5](imgs/lenet5.jpg)

In [ ]:
model = Sequential()

# model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(3,32,32)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=84, activation='relu'))
model.add(Dense(units=10, activation = 'softmax'))

epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))